In [92]:
from metafid.deriv import Pricing
from metafid.deriv import OptionStrategy
import pandas as pd
import jdatetime
import requests
from collections import namedtuple
from itertools import combinations
from operator import add

In [89]:
now = jdatetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'1401-12-28 12:32:27'

In [2]:
today = jdatetime.date.today().strftime("%Y-%m-%d")
today

'1401-12-28'

In [3]:
call = pd.read_excel("call.xlsx")
put = pd.read_excel("put.xlsx")

In [4]:
bsm = Pricing()
o_strategy = OptionStrategy()

In [5]:
bs_call = bsm.bs(call)
bs_call.bs = bs_call.bs.round(1)

Please Enetr a Valid Ticker or Name!


/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/option.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  d_1 = add(np.log(s / k), (self.r + mul(truediv(pow(sigma, 2), 2), t))) / (sigma * np.sqrt(t))


In [6]:
columns = ["j_date", "ticker","o_ticker", "t", "bs", "strike_price"] + [i for i in bs_call.columns if "adj" in i]
df_bs_call = bs_call[columns][bs_call.o_adj_final > 0]

In [7]:
bs_put = bsm.bs(put)

Please Enetr a Valid Ticker or Name!


In [8]:
df_bs_put = bs_put[columns][bs_put.o_adj_final > 0]
df_bs_put.bs = df_bs_put.bs.round(1)
df_bs_put.tail()

,j_date,ticker,o_ticker,t,bs,strike_price,adj_open,adj_high,adj_low,adj_close,adj_final,o_adj_open,o_adj_high,o_adj_low,o_adj_close,o_adj_final
126998,1401-12-06,خگستر,طستر2010,78,919.8,5500.0,4140,4308,4140,4308,4272,999.0,999.0,999.0,999.0,999.0
127034,1401-12-10,خگستر,طستر2008,74,369.6,4500.0,4289,4289,3993,3993,4077,500.0,500.0,500.0,500.0,500.0
127067,1401-12-15,خگستر,طستر2008,69,296.8,4500.0,4198,4300,4124,4184,4209,500.0,500.0,500.0,500.0,500.0
127111,1401-12-22,خگستر,طستر2008,62,337.7,4500.0,4090,4160,4035,4049,4080,700.0,700.0,700.0,700.0,700.0
131420,1401-12-10,شبندر,طبدر2042,60,1683.7,13000.0,10680,10680,10010,10150,10340,2500.0,2500.0,2500.0,2500.0,2500.0


### `covered call strategy`

In [9]:
df_covered_call = o_strategy.covered_call(df_bs_call)
df_covered_call.to_excel("rep_covered_call.xlsx")

### `married put strategy`

In [10]:
df_married_put = o_strategy.married_put(df_bs_put)
df_married_put.to_excel("rep_married_put.xlsx")

### `bull call spread strategy`

In [18]:
bull_call_spread = o_strategy.bull_call_spread(bs_call[bs_call.j_date=="1401-12-27"])
bull_call_spread.to_excel("rep_bull_call_spread.xlsx")
bull_call_spread.head()

,o_ticker,strike_price,t,bs,adj_final,o_adj_final,max_ptnl_loss,max_ptnl_profit,current_profit
0,"(ضهرم7016, ضهرم7015)","(26000.0, 24000.0)",211,"(3615.9, 4351.5)",21910,"(5289.0, 6628.0)",-1339.0,661.0,-1339.0
1,"(ضهرم7016, ضهرم7014)","(26000.0, 22000.0)",211,"(3615.9, 5215.4)",21910,"(5289.0, 7683.0)",-2394.0,1606.0,-2394.0
2,"(ضهرم7016, ضهرم7013)","(26000.0, 20000.0)",211,"(3615.9, 6218.9)",21910,"(5289.0, 8280.0)",-2991.0,3009.0,-1081.0
3,"(ضهرم7016, ضهرم7012)","(26000.0, 18000.0)",211,"(3615.9, 7369.0)",21910,"(5289.0, 9374.0)",-4085.0,3915.0,-175.0
4,"(ضهرم7016, ضهرم7010)","(26000.0, 15000.0)",211,"(3615.9, 9367.4)",21910,"(5289.0, 10227.0)",-4938.0,6062.0,1972.0


In [12]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

### `bear call spread strategy`

In [16]:
bear_call_spread = o_strategy.bear_call_spread(bs_call[bs_call.j_date==today])
bear_call_spread.to_excel("rep_bear_call_spread.xlsx")
bear_call_spread.head()

""


In [90]:
def mw():
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    r = requests.get(
        "http://www.tsetmc.com/tsev2/data/MarketWatchPlus.aspx", headers=headers
    )
    main_text = r.text
    ob_df = pd.DataFrame((main_text.split("@")[3]).split(";"))
    ob_df = ob_df[0].str.split(",", expand=True)
    ob_df.columns = [
        "web_id",
        "ob_depth",
        "sell_no",
        "buy_no",
        "buy_price",
        "sell_price",
        "buy_vol",
        "sell_vol",
    ]
    ob_df = ob_df[
        [
            "web_id",
            "ob_depth",
            "sell_no",
            "sell_vol",
            "sell_price",
            "buy_price",
            "buy_vol",
            "buy_no",
        ]
    ]
    ob_df.set_index("web_id", inplace=True)

    mw_df = pd.DataFrame((main_text.split("@")[2]).split(";"))
    mw_df = mw_df[0].str.split(",", expand=True)
    mw_df = mw_df.iloc[:, :23]
    mw_df.columns = [
        "web_id",
        "isin",
        "ticker",
        "name",
        "time",
        "open",
        "final",
        "close",
        "no",
        "volume",
        "value",
        "low",
        "high",
        "y_final",
        "eps",
        "base_vol",
        "unknown1",
        "unknown2",
        "sector",
        "day_ul",
        "day_ll",
        "share_no",
        "mkt_id",
    ]
    mw_df.set_index("web_id", inplace=True)
    df = mw_df.join(ob_df)
    df = df.assign(dt=jdatetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    return df

In [91]:
mw()

,isin,ticker,name,time,open,final,close,no,volume,value,...,share_no,mkt_id,ob_depth,sell_no,sell_vol,sell_price,buy_price,buy_vol,buy_no,dt
web_id,,,,,,,,,,,,,,,,,,,,,
10024128313803797,IRO3GDSZ0001,زگلدشت,كشت و دام گلدشت نمونه اصفهان,122959,18780,18750,19200,477,2953273,55370850730,...,700000000,303,2,1,300,19300,19160,77224,1,1401-12-28 12:37:19
10024128313803797,IRO3GDSZ0001,زگلدشت,كشت و دام گلدشت نمونه اصفهان,122959,18780,18750,19200,477,2953273,55370850730,...,700000000,303,3,2,1300,19340,19010,100324,2,1401-12-28 12:37:19
10024128313803797,IRO3GDSZ0001,زگلدشت,كشت و دام گلدشت نمونه اصفهان,122959,18780,18750,19200,477,2953273,55370850730,...,700000000,303,4,5,19128,19400,19000,128185,2,1401-12-28 12:37:19
10024128313803797,IRO3GDSZ0001,زگلدشت,كشت و دام گلدشت نمونه اصفهان,122959,18780,18750,19200,477,2953273,55370850730,...,700000000,303,5,5,21466,19500,18910,100000,1,1401-12-28 12:37:19
10024128313803797,IRO3GDSZ0001,زگلدشت,كشت و دام گلدشت نمونه اصفهان,122959,18780,18750,19200,477,2953273,55370850730,...,700000000,303,1,2,20272,19280,19200,17817,1,1401-12-28 12:37:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988953834777625,IRO9MSMI6541,ضملي7020,اختيارخ فملي-2333-1402/07/05,122941,5700,5689,5750,82,1777,15163123500,...,1500,311,2,1,50,5800,5620,1,1,1401-12-28 12:37:19
9988953834777625,IRO9MSMI6541,ضملي7020,اختيارخ فملي-2333-1402/07/05,122941,5700,5689,5750,82,1777,15163123500,...,1500,311,3,1,27,6000,5610,100,1,1401-12-28 12:37:19
9988953834777625,IRO9MSMI6541,ضملي7020,اختيارخ فملي-2333-1402/07/05,122941,5700,5689,5750,82,1777,15163123500,...,1500,311,4,1,100,6500,5601,10,1,1401-12-28 12:37:19
